# Who am I ?

My name is *Pierre Navaro*

I grew up in Normandy, France.

Ph.D in Computational Aeroacoustics, 1998-2002 (Université du Havre)

Scientific Software Engineer in Strasbourg (2003-2015)

Moved to Rennes in 2015

Skills : Python and Fortran with parallel programming.

Julia user since July 2018 (Julia v1.0)

# Why should i do a package ?

- Share your code and maintain it.
- Improve your programming pratices.
- Continuous integration is easier (Github+Travis-CI).
- Documentation can be generated automatically.
- Standard source files organization.

# Configure git

```bash
git config --global user.name "Pierre Navaro"
git config --global user.email "pierre.navaro@math.cnrs.fr"
git config --global github.user "pnavaro"
```

# Create the Julia package LyonCalcul

```julia
(v1.0) pkg> generate LyonCalcul
Generating project LyonCalcul:
    LyonCalcul/Project.toml
    LyonCalcul/src/LyonCalcul.jl
```

```julia
shell> cat LyonCalcul/Project.toml
authors = ["Pierre Navaro <pierre.navaro@math.cnrs.fr>"]
name = "LyonCalcul"
uuid = "417a5b38-18da-11e9-35ce-9bdc85ad86c9"
version = "0.1.0"

[deps]
```

```julia
shell> cat LyonCalcul/src/LyonCalcul.jl
module LyonCalcul

greet() = print("Hello World!")

end # module
```

# The LyonCalcul.jl module

In [20]:
module LyonCalcul

export bspline

"""
    bspline(p, j, x)

Return the value at x in [0,1[ of the B-spline with integer nodes of degree p with support starting at j.
Implemented recursively using the [De Boor's Algorithm](https://en.wikipedia.org/wiki/De_Boor%27s_algorithm)

```math
B_{i,0}(x) := \\left\\{
\\begin{matrix}
1 & \\mathrm{if}  \\quad t_i ≤ x < t_{i+1} \\\\
0 & \\mathrm{otherwise} 
\\end{matrix}
\\right.
```

```math
B_{i,p}(x) := \\frac{x - t_i}{t_{i+p} - t_i} B_{i,p-1}(x) 
+ \\frac{t_{i+p+1} - x}{t_{i+p+1} - t_{i+1}} B_{i+1,p-1}(x).
```
"""
function bspline(p::Int, j::Int, x::Float64)
   if p == 0
       if j == 0
           return 1.0
       else
           return 0.0
       end
   else
       w = (x - j) / p
       w1 = (x - j - 1) / p
   end
   return (w * bspline(p - 1, j, x) + (1 - w1) * bspline(p - 1, j + 1, x))
end

end # module

Main.LyonCalcul

# Activate your package

```julia
(v1.0) pkg> activate LyonCalcul

(LyonCalcul) pkg> instantiate
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/JuliaProjects/LyonCalcul/Project.toml`
 [no changes]
```

# Add some dependencies
```julia
(LyonCalcul) pkg> add FFTW
 Resolving package versions...
  Updating `~/JuliaProjects/LyonCalcul/Project.toml`
  [7a1cc6ca] + FFTW v0.2.4
  Updating `~/JuliaProjects/LyonCalcul/Manifest.toml`
  [621f4979] + AbstractFFTs v0.3.2
  [b99e7846] + BinaryProvider v0.5.3
  [34da2185] + Compat v1.4.0
  [8f4d0f93] + Conda v1.1.1
  ...
```

" The Manifest.toml allows someone to replicate the exact version of the dependencies that was recorded in the manifest on e.g. another machine. For a package that is to be used as a library, this is not useful.

However, for an “application”, i.e. something at “top level” (say your julia code to do the simulations in a scientific paper) then it is likely useful to be able to replicate that exact state and the Manifest is thus useful to check in."


*Kristoffer Carlsson* (Julia Computing)

# Check the documentation

```julia
julia> using LyonCalcul

help?> bspline
search: bspline

  bspline(p, j, x)

  Return the value at x in [0,1[ of the B-spline with integer nodes of degree p with support starting at j.
  Implemented recursively using the De Boor's Algorithm (https://en.wikipedia.org/wiki/De_Boor%27s_algorithm)

B_{i,0}(x) := \left\{
\begin{matrix}
1 & \mathrm{if}  \quad t_i ≤ x < t_{i+1} \\
0 & \mathrm{otherwise}
\end{matrix}
\right.

B_{i,p}(x) := \frac{x - t_i}{t_{i+p} - t_i} B_{i,p-1}(x)
+ \frac{t_{i+p+1} - x}{t_{i+p+1} - t_{i+1}} B_{i+1,p-1}(x).
```

In [23]:
?LyonCalcul.bspline

```
bspline(p, j, x)
```

Return the value at x in [0,1[ of the B-spline with integer nodes of degree p with support starting at j. Implemented recursively using the [De Boor's Algorithm](https://en.wikipedia.org/wiki/De_Boor%27s_algorithm)

$$
B_{i,0}(x) := \left\{
\begin{matrix}
1 & \mathrm{if}  \quad t_i ≤ x < t_{i+1} \\
0 & \mathrm{otherwise} 
\end{matrix}
\right.
$$

$$
B_{i,p}(x) := \frac{x - t_i}{t_{i+p} - t_i} B_{i,p-1}(x) 
+ \frac{t_{i+p+1} - x}{t_{i+p+1} - t_{i+1}} B_{i+1,p-1}(x).
$$


# Add a test

```bash
cd LyonCalcul
mkdir test
```

add file runtests.jl

```julia
shell> cat LyonCalcul/test/runtests.jl
using Test
using LyonCalcul

@testset "Test bspline function" begin

    for p in 1:5
        @test sum([bspline(p, j, 0.5) for j in -p:0]) ≈ 1.0 atol = 1e-16
    end
end

```

## Verify the test

```julia
(LyonCalcul) pkg> add Test
 Resolving package versions...
  Updating `~/JuliaProjects/LyonCalcul/Project.toml`
  [8dfed614] + Test
  Updating `~/JuliaProjects/LyonCalcul/Manifest.toml`
 [no changes]

(LyonCalcul) pkg> test
   Testing LyonCalcul
 Resolving package versions...
Test Summary:         | Pass  Total
Test bspline function |    5      5
   Testing LyonCalcul tests passed
```

In [24]:
"""
    interpolate( p, f, delta, alpha)

Compute the interpolating spline of degree p of odd
degree of a 1D function f on a periodic uniform mesh, at
all points x-alpha. f type is Vector{Float64}.

Derived from a Python program written by
Eric Sonnendrucker (Max-Planck-Institut fur Plasmaphysik - Garching)

"""
function interpolate(p     :: Int,
                     f     :: Vector{Float64},
                     delta :: Float64,
                     alpha :: Float64)

   n = size(f)[1]
   modes = 2π * (0:n-1) / n
   eig_bspl = zeros(Complex{Float64},n)
   eig_bspl .= bspline(p, -div(p+1,2), 0.0)
   for j in 1:div(p+1,2)-1
      eig_bspl .+= (bspline(p, j-div(p+1,2), 0.0)
         * 2 * cos.(j * modes))
   end
   ishift = floor(- alpha / delta)
   beta = - ishift - alpha / delta
   eigalpha = zeros(Complex{Float64},n)
   for j in -div(p-1,2):div(p+1,2)
      eigalpha .+= (bspline(p, j-div(p+1,2), beta)
         .* exp.((ishift + j) * 1im .* modes))
   end

   real(ifft(fft(f) .* eigalpha ./ eig_bspl))

end

interpolate

In [30]:
using Plots, LyonCalcul
gr()
xmin, xmax, nx = 0, 4π, 128
x = range(xmin, stop=xmax, length=nx+1)[1:end-1] |> collect
f = sin.(x) .- cos.(2x)
plot(x, f)
p = 5
dx = (xmax - xmin) / nx
alpha = 1.0
plot(x, interpolate(5, f, dx, alpha))

ArgumentError: ArgumentError: Package LyonCalcul not found in current path:
- Run `import Pkg; Pkg.add("LyonCalcul")` to install the LyonCalcul package.


- Create docs directory
- add docs/src/index.md file with

~~~
# Example.jl Documentation

## Types and Functions

```@autodocs
Modules = [Example]
Order   = [:type, :function]
```

~~~

## Documentation

https://github.com/JuliaDocs/Documenter.jl

- Push changes to github
- add docs/build in .gitignore
- add Manifest.toml also




- Create `docs/make.jl` file

```julia

using Example
using Documenter

makedocs(modules=[Example],
         doctest = false,
         format = :html,
         sitename = "Example.jl",
         pages = ["Documentation"    => "index.md"])

deploydocs(
    deps   = Deps.pip("mkdocs", "python-markdown-math"),
    repo   = "github.com/pnavaro/Example.jl.git",
 )
```

- Install first version of Example package in your julia installation

```julia
(v1.0) pkg> add https://github.com/pnavaro/Example.jl.git
```

- Test it
```bash
cd docs
julia make.jl
```

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*

# Add a repository on Github

https://github.com/pnavaro/LyonCalcul.jl

- The repository name has the ".jl" extension

# Travis

https://travis-ci.org

- Profile -> Settings
-

# Push the package on github

```bash
cd LyonCalcul
echo "# LyonCalcul.jl" >> README.md
git init
git add README.md
git commit -m "first commit"
git remote add origin git@github.com:pnavaro/LyonCalcul.jl.git
git push -u origin master
```

# On Github choose your license

Above the file list, click Create new file.

In the file name field, type LICENSE (with all caps).

- Choose a license template button.
- Click Choose a license template.
- Add a license to your project.
- Don't create pull request choose "master" branch.

# On your computer

```bash
git pull origin master
```